In [ ]:
pip install git+https://github.com/google-research/bleurt

  Cloning https://github.com/google-research/bleurt to /tmp/pip-req-build-ankbm6rl
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt /tmp/pip-req-build-ankbm6rl
  Resolved https://github.com/google-research/bleurt to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456765 sha256=f279c1fe15700a8889ae6033288effe6e5ed8b9267472e732cbb536a4ecf6f17
  Stored in directory: /tmp/pip-ephem-wheel-cache-hm5c_pv4/wheels/63/3b/9f/72787ebc77cdf550411b393adea50e0272b9c5bb857f81f4b0
Successfully built BLEURT


In [ ]:
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
!unzip BLEURT-20.zip

--2024-02-02 16:51:06--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2140294207 (2.0G) [application/octet-stream]
Saving to: ‘BLEURT-20.zip’

BLEURT-20.zip       100%[===================>]   1.99G   182MB/s    in 18s     

2024-02-02 16:51:25 (112 MB/s) - ‘BLEURT-20.zip’ saved [2140294207/2140294207]

--2024-02-02 16:51:25--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2024-02-02 16:51:25--
Total wall clock time: 18s
Downloaded: 1 files, 2.0G in 18s (112 MB/s)
Archive:  BLEURT-20.zip
   creating: BLEURT-20/
  inflating: BLEURT-20/bert_config.json  
  inflating: BLEURT-20/saved_model.pb  
   creating: BLEURT-20/variables/
  inflat

In [ ]:
!wget https://drive.switch.ch/index.php/s/fKMKOmvUU143S2y/download
!mv download wmt22_translations.tar.gz

--2024-02-02 16:51:49--  https://drive.switch.ch/index.php/s/fKMKOmvUU143S2y/download
Resolving drive.switch.ch (drive.switch.ch)... 86.119.34.137, 86.119.34.138, 2001:620:5ca1:1ee::12, ...
Connecting to drive.switch.ch (drive.switch.ch)|86.119.34.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5209639 (5.0M) [application/gzip]
Saving to: ‘download’

download            100%[===================>]   4.97M  3.99MB/s    in 1.2s    

2024-02-02 16:51:51 (3.99 MB/s) - ‘download’ saved [5209639/5209639]



In [ ]:
!tar -xzvf wmt22_translations.tar.gz

mbr.wmt22.de-en.aggregate.n1024.epsilon0.02.seed0.chrf.en
mbr.wmt22.de-en.aggregate.n1024.epsilon0.02.seed0.comet22.en
mbr.wmt22.de-en.aggregate.n1024.epsilon0.02.seed0.cometinho.en
mbr.wmt22.de-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf.en
mbr.wmt22.de-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.en
mbr.wmt22.de-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.comet22.en
mbr.wmt22.de-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.cometinho.en
mbr.wmt22.de-en.coarse_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.en
mbr.wmt22.de-en.pairwise.n1024.epsilon0.02.seed0.chrf.en
mbr.wmt22.de-en.pairwise.n1024.epsilon0.02.seed0.comet22.en
mbr.wmt22.de-en.pairwise.n1024.epsilon0.02.seed0.cometinho.en
mbr.wmt22.en-de.aggregate.n1024.epsilon0.02.seed0.chrf.de
mbr.wmt22.en-de.aggregate.n1024.epsilon0.02.seed0.comet22.de
mbr.wmt22.en-de.aggregate.n1024.epsilon0.02.seed0.cometinho.de
mbr.wmt22.en-de.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf.d

In [ ]:
from pathlib import Path
import numpy as np
from bleurt import score

def run_score(r_path, t_path):
  checkpoint = "BLEURT-20"
  references = Path(r_path).read_text().splitlines()
  candidates = Path(t_path).read_text().splitlines()
  assert len(references) == len(candidates)

  scorer = score.BleurtScorer(checkpoint)
  scores = scorer.score(references=references, candidates=candidates)
  avg = np.mean(scores)
  print(f"{t_path}\t{checkpoint}\t{avg:.5f}")

In [ ]:
for lp in ["en-de", "de-en", "en-ru", "ru-en"]:
  src = lp.split("-")[0]
  tgt = lp.split("-")[1]
  r_path = f"wmt22.{lp}.ref.{tgt}"
  t_path = f"wmt22.{lp}.beam4.{tgt}"
  run_score(r_path, t_path)

wmt22.en-de.beam4.de	BLEURT-20	0.76158
wmt22.de-en.beam4.en	BLEURT-20	0.72562
wmt22.en-ru.beam4.ru	BLEURT-20	0.68498
wmt22.ru-en.beam4.en	BLEURT-20	0.75474


In [ ]:
for lp in ["en-de", "de-en", "en-ru", "ru-en"]:
  src = lp.split("-")[0]
  tgt = lp.split("-")[1]
  r_path = f"wmt22.{lp}.ref.{tgt}"
  t_path = f"wmt22.{lp}.epsilon0.02.seed0.{tgt}"
  run_score(r_path, t_path)

wmt22.en-de.epsilon0.02.seed0.de	BLEURT-20	0.73392
wmt22.de-en.epsilon0.02.seed0.en	BLEURT-20	0.69702
wmt22.en-ru.epsilon0.02.seed0.ru	BLEURT-20	0.65791
wmt22.ru-en.epsilon0.02.seed0.en	BLEURT-20	0.72133


In [ ]:
for lp in ["en-de", "de-en", "en-ru", "ru-en"]:
  src = lp.split("-")[0]
  tgt = lp.split("-")[1]
  r_path = f"wmt22.{lp}.ref.{tgt}"
  run_score(r_path, f"mbr.wmt22.{lp}.pairwise.n1024.epsilon0.02.seed0.chrf.{tgt}")
  run_score(r_path, f"mbr.wmt22.{lp}.aggregate.n1024.epsilon0.02.seed0.chrf.{tgt}")
  run_score(r_path, f"mbr.wmt22.{lp}.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf.{tgt}")

mbr.wmt22.en-de.pairwise..n1024.epsilon0.02.seed0.chrf.de	BLEURT-20	0.76032
mbr.wmt22.en-de.aggregate.n1024.epsilon0.02.seed0.chrf.de	BLEURT-20	0.75953
mbr.wmt22.en-de.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf.de	BLEURT-20	0.76020
mbr.wmt22.de-en.pairwise..n1024.epsilon0.02.seed0.chrf.en	BLEURT-20	0.72725
mbr.wmt22.de-en.aggregate.n1024.epsilon0.02.seed0.chrf.en	BLEURT-20	0.72791
mbr.wmt22.de-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf.en	BLEURT-20	0.72801
mbr.wmt22.en-ru.pairwise..n1024.epsilon0.02.seed0.chrf.ru	BLEURT-20	0.69516
mbr.wmt22.en-ru.aggregate.n1024.epsilon0.02.seed0.chrf.ru	BLEURT-20	0.69463
mbr.wmt22.en-ru.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf.ru	BLEURT-20	0.69537
mbr.wmt22.ru-en.pairwise..n1024.epsilon0.02.seed0.chrf.en	BLEURT-20	0.75506
mbr.wmt22.ru-en.aggregate.n1024.epsilon0.02.seed0.chrf.en	BLEURT-20	0.75445
mbr.wmt22.ru-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf.en	BLEURT-20	0.75469


In [ ]:
for lp in ["en-de", "de-en", "en-ru", "ru-en"]:
  src = lp.split("-")[0]
  tgt = lp.split("-")[1]
  r_path = f"wmt22.{lp}.ref.{tgt}"
  run_score(r_path, f"mbr.wmt22.{lp}.pairwise.n1024.epsilon0.02.seed0.cometinho.{tgt}")
  run_score(r_path, f"mbr.wmt22.{lp}.aggregate.n1024.epsilon0.02.seed0.cometinho.{tgt}")
  run_score(r_path, f"mbr.wmt22.{lp}.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.cometinho.{tgt}")

mbr.wmt22.en-de.pairwise..n1024.epsilon0.02.seed0.cometinho.de	BLEURT-20	0.76667
mbr.wmt22.en-de.aggregate..n1024.epsilon0.02.seed0.cometinho.de	BLEURT-20	0.76558
mbr.wmt22.en-de.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.cometinho.de	BLEURT-20	0.76645
mbr.wmt22.de-en.pairwise..n1024.epsilon0.02.seed0.cometinho.en	BLEURT-20	0.73054
mbr.wmt22.de-en.aggregate..n1024.epsilon0.02.seed0.cometinho.en	BLEURT-20	0.72863
mbr.wmt22.de-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.cometinho.en	BLEURT-20	0.72968
mbr.wmt22.en-ru.pairwise..n1024.epsilon0.02.seed0.cometinho.ru	BLEURT-20	0.70571
mbr.wmt22.en-ru.aggregate..n1024.epsilon0.02.seed0.cometinho.ru	BLEURT-20	0.70394
mbr.wmt22.en-ru.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.cometinho.ru	BLEURT-20	0.70556
mbr.wmt22.ru-en.pairwise..n1024.epsilon0.02.seed0.cometinho.en	BLEURT-20	0.75905
mbr.wmt22.ru-en.aggregate..n1024.epsilon0.02.seed0.cometinho.en	BLEURT-20	0.75736
mbr.wmt22.ru-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0

In [ ]:
for lp in ["en-de", "de-en", "en-ru", "ru-en"]:
  src = lp.split("-")[0]
  tgt = lp.split("-")[1]
  r_path = f"wmt22.{lp}.ref.{tgt}"
  run_score(r_path, f"mbr.wmt22.{lp}.pairwise.n1024.epsilon0.02.seed0.comet22.{tgt}")
  run_score(r_path, f"mbr.wmt22.{lp}.aggregate.n1024.epsilon0.02.seed0.comet22.{tgt}")
  run_score(r_path, f"mbr.wmt22.{lp}.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.comet22.{tgt}")

mbr.wmt22.en-ru.pairwise..n1024.epsilon0.02.seed0.comet22.ru	BLEURT-20	0.72401
mbr.wmt22.en-ru.aggregate..n1024.epsilon0.02.seed0.comet22.ru	BLEURT-20	0.72054
mbr.wmt22.en-ru.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.comet22.ru	BLEURT-20	0.72294
mbr.wmt22.ru-en.pairwise..n1024.epsilon0.02.seed0.comet22.en	BLEURT-20	0.76110
mbr.wmt22.ru-en.aggregate..n1024.epsilon0.02.seed0.comet22.en	BLEURT-20	0.76054
mbr.wmt22.ru-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.comet22.en	BLEURT-20	0.76130


In [ ]:
for lp in ["en-de", "de-en", "en-ru", "ru-en"]:
  src = lp.split("-")[0]
  tgt = lp.split("-")[1]
  r_path = f"wmt22.{lp}.ref.{tgt}"
  run_score(r_path, f"mbr.wmt22.{lp}.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.{tgt}")
  run_score(r_path, f"mbr.wmt22.{lp}.coarse_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.{tgt}")

mbr.wmt22.en-de.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.de	BLEURT-20	0.76922
mbr.wmt22.en-de.coarse_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.de	BLEURT-20	0.76963
mbr.wmt22.de-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.en	BLEURT-20	0.73337
mbr.wmt22.de-en.coarse_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.en	BLEURT-20	0.73264
mbr.wmt22.en-ru.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.ru	BLEURT-20	0.71191
mbr.wmt22.en-ru.coarse_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.ru	BLEURT-20	0.71172
mbr.wmt22.ru-en.aggregate_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.en	BLEURT-20	0.76104
mbr.wmt22.ru-en.coarse_to_fine.top20.n1024.epsilon0.02.seed0.chrf-to-comet22.en	BLEURT-20	0.76154
